Импорт необходимых библиотек

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from lightgbm import LGBMClassifier, Dataset

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

pd.options.display.max_columns=100

Загрузка обучающего и тестового датасетов

In [2]:
train = pd.read_csv('data/catboost_train.csv')
test = pd.read_csv('data/catboost_test.csv')

In [3]:
cat_cols = [
    'purpose',
    'home_ownership',
    'subgrade'
]

for cat_cols in cat_cols:
    le = LabelEncoder()
    train[cat_cols] = le.fit_transform(train[cat_cols])
    test[cat_cols] = le.transform(test[cat_cols])

In [4]:
targ_cols = [i for i in train.columns if i not in ['def', 'id'] ]

X_train = train[targ_cols]
X_test = test[targ_cols]

y_train = train["def"]

In [5]:
targ_cols

['installment',
 'dti',
 'funded_amnt',
 'annual_inc',
 'emp_length',
 'term',
 'inq_last_6mths',
 'mths_since_recent_inq',
 'delinq_2yrs',
 'chargeoff_within_12_mths',
 'num_accts_ever_120_pd',
 'num_tl_90g_dpd_24m',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'tot_hi_cred_lim',
 'delinq_amnt',
 'purpose',
 'subgrade',
 'home_ownership',
 'psk',
 'percent',
 'percent_per_year',
 'residual_dti',
 'loan_income_ratio',
 'additional_dti',
 'dti_ratio_min',
 'dti_ratio_max',
 'waste_to_installment_ratio',
 'is_first_loan',
 'is_not_sber_client',
 'even_salary',
 'even_installment',
 'is_emp_length_unknown',
 'is_dti_b_35',
 'is_common_amount',
 'funded_amnt_mod_1000',
 'funded_amnt_mod_500',
 'funded_amnt_mod_100',
 'funded_amnt_mod_10']

In [22]:
lgb_params = {
    'objective': 'binary',
    'max_depth': 1,
#     'num_leaves': 4,
    'learning_rate': 0.1,
#     'lambda_l1': 12,
    'seed': 17,
    'n_estimators': 1000,
    'min_child_weight': 40,
    'min_gain_to_split': 1
}

columns = [
    'percent',
    'acc_open_past_24mths',
    'avg_cur_bal',
    'loan_income_ratio',
    'annual_inc',
    'tot_hi_cred_lim',
    'additional_dti',
    'percent_per_year',
    'purpose',
    'dti',
    'home_ownership',
    'num_accts_ever_120_pd',
    'psk',
    'is_emp_length_unknown',
    'funded_amnt',
]

cat_features = [
    'purpose',
    'home_ownership',
]

skf = StratifiedKFold(n_splits=5, random_state=109)
oof_result = pd.Series([0] * train.shape[0])
result = [0] * test.shape[0]
scores = []
for ind_trn, ind_test in skf.split(X_train, y_train):
    X_train_df = X_train[columns].iloc[ind_trn]
    X_valid_df = X_train[columns].iloc[ind_test]
    y_train_df = y_train.iloc[ind_trn]
    y_valid_df = y_train.iloc[ind_test]

    model = LGBMClassifier(**lgb_params)

    model.fit(
        X_train_df,  
        y_train_df,
        eval_set=[(X_valid_df, y_valid_df)],
        eval_metric='auc',
        verbose=100,
        early_stopping_rounds=50,
        categorical_feature=cat_features,
    )
    result += model.predict_proba(test[columns])[:, 1:].flatten()/5
    scores.append(model.best_score_['valid_0']['auc'])
    oof_result.loc[ind_test] = model.predict_proba(X_valid_df)[:, 1]


c:\users\kozlo\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
c:\users\kozlo\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:1555: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['home_ownership', 'purpose']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
c:\users\kozlo\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
c:\users\kozlo\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Warning] min_gain_to_split is set=1, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=1
Training until validation scores don't improve for 50 rounds
[100]	valid_0's auc: 0.689865	valid_0's binary_logloss: 0.41141
[200]	valid_0's auc: 0.693258	valid_0's binary_logloss: 0.409819
[300]	valid_0's auc: 0.693495	valid_0's binary_logloss: 0.409573
Early stopping, best iteration is:
[276]	valid_0's auc: 0.693685	valid_0's binary_logloss: 0.409533


c:\users\kozlo\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
c:\users\kozlo\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:1555: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['home_ownership', 'purpose']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
c:\users\kozlo\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
c:\users\kozlo\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Warning] min_gain_to_split is set=1, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=1
Training until validation scores don't improve for 50 rounds
[100]	valid_0's auc: 0.684295	valid_0's binary_logloss: 0.41207
[200]	valid_0's auc: 0.687493	valid_0's binary_logloss: 0.410474
[300]	valid_0's auc: 0.688105	valid_0's binary_logloss: 0.410221
Early stopping, best iteration is:
[331]	valid_0's auc: 0.688227	valid_0's binary_logloss: 0.410187


c:\users\kozlo\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
c:\users\kozlo\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:1555: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['home_ownership', 'purpose']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
c:\users\kozlo\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
c:\users\kozlo\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Warning] min_gain_to_split is set=1, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=1
Training until validation scores don't improve for 50 rounds
[100]	valid_0's auc: 0.683619	valid_0's binary_logloss: 0.412763
[200]	valid_0's auc: 0.68805	valid_0's binary_logloss: 0.410925
[300]	valid_0's auc: 0.688603	valid_0's binary_logloss: 0.410648
Early stopping, best iteration is:
[293]	valid_0's auc: 0.688674	valid_0's binary_logloss: 0.410623
[LightGBM] [Warning] min_gain_to_split is set=1, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=1
Training until validation scores don't improve for 50 rounds


c:\users\kozlo\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
c:\users\kozlo\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:1555: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['home_ownership', 'purpose']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
c:\users\kozlo\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
c:\users\kozlo\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[100]	valid_0's auc: 0.679021	valid_0's binary_logloss: 0.413093
[200]	valid_0's auc: 0.681875	valid_0's binary_logloss: 0.411984
[300]	valid_0's auc: 0.682699	valid_0's binary_logloss: 0.411771
Early stopping, best iteration is:
[340]	valid_0's auc: 0.682958	valid_0's binary_logloss: 0.411725


c:\users\kozlo\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
c:\users\kozlo\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:1555: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['home_ownership', 'purpose']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
c:\users\kozlo\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
c:\users\kozlo\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Warning] min_gain_to_split is set=1, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=1
Training until validation scores don't improve for 50 rounds
[100]	valid_0's auc: 0.673939	valid_0's binary_logloss: 0.415458
[200]	valid_0's auc: 0.67671	valid_0's binary_logloss: 0.414747
Early stopping, best iteration is:
[205]	valid_0's auc: 0.676956	valid_0's binary_logloss: 0.414665


In [23]:
#depth 4
np.mean(scores), np.std(scores), roc_auc_score(y_train, oof_result)

(0.6861002297092291, 0.005694943259212901, 0.6857824384945757)

Применение модели к тестовой выборке

In [24]:
test['prob'] = result

Выгрузка решения для коммита в соревновании

In [25]:
test[['id', 'prob']].to_csv('lgbm_submission.csv', index=False)